In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation, Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from imutils import paths
from sklearn.model_selection import train_test_split

In [27]:
dir_path = '/content/drive/MyDrive/archive/seg_train/seg_train'
categories = ['street','mountain','sea']
data = []
N = 500 #500장씩만 사용

for i in categories:

    paths = os.path.join(dir_path,i)
    print('paths',paths)
    count = 0
    for j in os.listdir(paths):
        img_path = os.path.join(paths, j)
        labels = categories.index(i)
        ext = os.path.splitext(img_path)[1].lower()
        if ext in ['.jpg', '.jpeg', '.png']:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (32, 32))
                data.append([img, labels])
                count += 1
            if count >= N:
                break  # 500장만 추가

random.shuffle(data)
print(len(data))

paths /content/drive/MyDrive/archive/seg_train/seg_train/street
paths /content/drive/MyDrive/archive/seg_train/seg_train/mountain
paths /content/drive/MyDrive/archive/seg_train/seg_train/sea
1500


In [28]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)
x = x/255.0
x = np.array(x).reshape(-1, 32, 32, 3)
print("Shape of train images is:", x.shape)
print("Shape of labels is:", y.shape)
print(y)
print(x.shape[1:])

Shape of train images is: (1500, 32, 32, 3)
Shape of labels is: (1500,)
[0 2 2 ... 1 0 1]
(32, 32, 3)


In [30]:
model = Sequential()
model.add(Conv2D(64,(3,3), activation='relu', input_shape = (x.shape[1:])))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,627 (697.76 KB)

 Trainable params: 178,627 (697.76 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x, y, epochs=50,batch_size=10,validation_split=0.2)

Epoch 1/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.3655 - loss: 1.0981 - val_accuracy: 0.5667 - val_loss: 0.8158
Epoch 2/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5874 - loss: 0.8254 - val_accuracy: 0.7167 - val_loss: 0.6647
Epoch 3/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.6496 - loss: 0.7018 - val_accuracy: 0.6600 - val_loss: 0.6364
Epoch 4/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.7033 - loss: 0.6590 - val_accuracy: 0.7567 - val_loss: 0.5803
Epoch 5/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7343 - loss: 0.5874 - val_accuracy: 0.7767 - val_loss: 0.5343
Epoch 6/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7520 - loss: 0.5598 - val_accuracy: 0.7633 - val_loss: 0.5619
Epoch 7/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7897 - loss: 0.5010 - val_accuracy: 0.7633 - val_loss: 0.5509
Epoch 8/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.7810 - loss: 0.5249 - val_accu

In [32]:

import numpy as np


def image(path):
    print('path',path)
    img = cv2.imread(path)

    new_arr = cv2.resize(img, (32, 32))
    new_arr = np.array(new_arr)
    new_arr = new_arr/255.0
    new_arr = new_arr.reshape(-1, 32, 32, 3)

    return new_arr



In [33]:
dir_path = '/content/drive/MyDrive/archive/seg_test/seg_test'
categories = ['street','mountain','sea']
data = []

correct_cnt=0
N = 20 #20장
for i in categories:
    paths = os.path.join(dir_path,i)
    files = os.listdir(paths)
    files = files[:N]  # 앞에서 N장만 사용
    for j in files:
        img_path = os.path.join(paths,j)
      #  print(img_path)
        labels = categories.index(i)
      #  print(img_path,labels)
        prediction = model.predict(image(img_path))
        print('labels' ,labels)
        print('classification results' ,prediction.argmax())
        if (labels==prediction.argmax()):
            correct_cnt=correct_cnt+1
print('### correnct cnt out of 20==>',correct_cnt)

path /content/drive/MyDrive/archive/seg_test/seg_test/street/20090.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20158.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20209.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20183.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20184.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20080.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
labels 0
classification results 0
path /content/drive/MyDrive/archive/seg_test/seg_test/street/20126.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
labels 0
classification results 0